In [1]:
import numpy as np
from tic_env import TictactoeEnv, OptimalPlayer

# Tic Toc Toe environment

Our 1st game is the famous Tic Toc Toe. You can read about the game and its rules here: https://en.wikipedia.org/wiki/Tic-tac-toe

We implemented the game as an environment in the style of games in the [Python GYM library](https://gym.openai.com/). The commented source code is available in the file "tic_env.py". Here, we give a brief introduction to the environment and how it can be used.

### Initialization and attributes

You can initialize the environment / game as following:

In [2]:
env = TictactoeEnv()

Which then has the following attributes with the corresponding initial values:

In [3]:
env.__dict__

{'grid': array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 'end': False,
 'winner': None,
 'player2value': {'X': 1, 'O': -1},
 'num_step': 0,
 'current_player': 'X'}

The game is played by two players: player 'X' and player 'O'. The attribute 'current_player' shows whose turn it is. We assume that player 'X' always plays first.

The attribute 'grid' is a 3x3 numpy array and presents the board in the real game and the state $s_t$ in the reinfocement learning language. Each elements can take a value in {0, 1, -1}:
     0 : place unmarked
     1 : place marked with X 
    -1 : place marked with O 
        
The attribute 'end' shows if the game is over or not, and the attribute 'winner' shows the winner of the game: either "X", "O", or None.  

You can use function 'render' to visualize the current position of the board:

In [4]:
env.render()

|- - -|
|- - -|
|- - -|



### Taking actions

The game environment will recieve action from two players in turn and update the grid. At each time, one player can take the action $a_t$, where $a_t$ can either be an integer between 0 to 8 or a touple, corresponding to the 9 possible.

Function 'step' is used to recieve the action of the player, update the grid:

In [5]:
env.step(2)

(array([[0., 0., 1.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 False,
 None)

In [6]:
env.render()

|- - X|
|- - -|
|- - -|



In [7]:
env.__dict__

{'grid': array([[0., 0., 1.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 'end': False,
 'winner': None,
 'player2value': {'X': 1, 'O': -1},
 'num_step': 1,
 'current_player': 'O'}

In [8]:
env.step((1,1))

(array([[ 0.,  0.,  1.],
        [ 0., -1.,  0.],
        [ 0.,  0.,  0.]]),
 False,
 None)

In [9]:
env.render()

|- - X|
|- O -|
|- - -|



In [10]:
env.__dict__

{'grid': array([[ 0.,  0.,  1.],
        [ 0., -1.,  0.],
        [ 0.,  0.,  0.]]),
 'end': False,
 'winner': None,
 'player2value': {'X': 1, 'O': -1},
 'num_step': 2,
 'current_player': 'X'}

But not all actions are available at each time: One cannot choose a place which has been taken before. There is an error if an unavailable action is taken:

In [11]:
env.step((0,2))

ValueError: There is already a chess on position (0, 2).

### Reward

Reward is always 0 until the end of the game. When the game is over, the reward is 1 if you win the game, -1 if you lose, and 0 besides. Function 'observe' can be used after each step to recieve the new state $s_t$, whether the game is over, and the winner, and function 'reward' to get the reward value $r_t$:

In [12]:
env.observe()

(array([[ 0.,  0.,  1.],
        [ 0., -1.,  0.],
        [ 0.,  0.,  0.]]),
 False,
 None)

In [13]:
env.reward(player='X')

0

In [14]:
env.reward(player='O')

0

An example of finishing the game:

In [15]:
env.step(0)
env.step(3)
env.step(1)

(array([[ 1.,  1.,  1.],
        [-1., -1.,  0.],
        [ 0.,  0.,  0.]]),
 True,
 'X')

In [16]:
env.render()

|X X X|
|O O -|
|- - -|



In [17]:
env.observe()

(array([[ 1.,  1.,  1.],
        [-1., -1.,  0.],
        [ 0.,  0.,  0.]]),
 True,
 'X')

In [18]:
env.reward(player='X')

1

In [19]:
env.reward(player='O')

-1

# Optimal policy for Tic Toc Toe environment

Fortunately, we know the exact optimal policy for Tic Toc Toe. We have implemented and $\epsilon$-greedy version of optimal polciy which you can use for the project.

In [20]:
env.reset();

In [21]:
opt_player = OptimalPlayer(epsilon = 0., player = 'X')

In [22]:
opt_player.act(env.grid)

(2, 2)

In [23]:
opt_player.player

'X'

### An example of optimal player playing against random player

In [24]:
Turns = np.array(['X','O'])
for i in range(5):
    env.reset()
    grid, _, __ = env.observe()
    Turns = Turns[np.random.permutation(2)]
    player_opt = OptimalPlayer(epsilon=0., player=Turns[0])
    player_rnd = OptimalPlayer(epsilon=1., player=Turns[1])
    for j in range(9):
        if env.current_player == player_opt.player:
            move = player_opt.act(grid)
        else:
            move = player_rnd.act(grid)

        grid, end, winner = env.step(move, print_grid=False)

        if end:
            print('-------------------------------------------')
            print('Game end, winner is player ' + str(winner))
            print('Optimal player = ' +  Turns[0])
            print('Random player = ' +  Turns[1])
            env.render()
            env.reset()
            break


-------------------------------------------
Game end, winner is player X
Optimal player = X
Random player = O
|O - -|
|X X X|
|O O X|

-------------------------------------------
Game end, winner is player X
Optimal player = X
Random player = O
|O O -|
|O X -|
|X X X|

-------------------------------------------
Game end, winner is player O
Optimal player = O
Random player = X
|X - O|
|- O -|
|O X X|

-------------------------------------------
Game end, winner is player X
Optimal player = X
Random player = O
|X O O|
|- X -|
|- - X|

-------------------------------------------
Game end, winner is player X
Optimal player = X
Random player = O
|- - X|
|O X -|
|X - O|



### An example of optimal player playing against optimal player

In [25]:
Turns = np.array(['X','O'])
for i in range(5):
    env.reset()
    grid, _, __ = env.observe()
    Turns = Turns[np.random.permutation(2)]
    player_opt_1 = OptimalPlayer(epsilon=0., player=Turns[0])
    player_opt_2 = OptimalPlayer(epsilon=0., player=Turns[1])
    for j in range(9):
        if env.current_player == player_opt.player:
            move = player_opt_1.act(grid)
        else:
            move = player_opt_2.act(grid)

        grid, end, winner = env.step(move, print_grid=False)

        if end:
            print('-------------------------------------------')
            print('Game end, winner is player ' + str(winner))
            print('Optimal player 1 = ' +  Turns[0])
            print('Optimal player 2 = ' +  Turns[1])
            env.render()
            env.reset()
            break


-------------------------------------------
Game end, winner is player None
Optimal player 1 = O
Optimal player 2 = X
|X O X|
|X O O|
|O X X|

-------------------------------------------
Game end, winner is player None
Optimal player 1 = O
Optimal player 2 = X
|X X O|
|O O X|
|X O X|

-------------------------------------------
Game end, winner is player None
Optimal player 1 = O
Optimal player 2 = X
|O X X|
|X O O|
|X O X|

-------------------------------------------
Game end, winner is player None
Optimal player 1 = X
Optimal player 2 = O
|X O X|
|O O X|
|X X O|

-------------------------------------------
Game end, winner is player None
Optimal player 1 = O
Optimal player 2 = X
|O X X|
|X X O|
|O O X|



## Q-learning 

In [81]:
from collections import defaultdict
from tqdm import tqdm 

coords = lambda pos: (int(pos / 3), pos % 3) 

def eps_greedy(Q, state, grid, epsilon):
    possible_moves = np.where(np.ravel(grid) == 0)[0]
    if np.random.random() < epsilon:
        position = np.random.choice(possible_moves)
    else:
        possible = [int(elem) for elem in np.argsort(Q[state]) if elem in possible_moves]
        position = possible[-1] if possible is not [] else np.random.choice(possible_moves) 
    return (int(position / 3), position % 3) 

#Q learning with greedy
#Dont necessariliy need to train with player_epsilon = 0.5
def q_learning(env, num_episodes, discount_factor=0.99, alpha=0.05, epsilon=0.3, player_epsilon=0.5):
    Q = defaultdict(lambda: np.zeros(9))
    episode_lengths = np.zeros(num_episodes)
    episode_rewards = np.zeros(num_episodes)
    Turns = np.array(['X','O'])
    
    for i_episode in tqdm(range(num_episodes)):
        env.reset()
        grid, end, winner = env.observe()
        Turns = Turns[np.random.permutation(2)]
        player = OptimalPlayer(epsilon=player_epsilon, player=Turns[1])
        while not end:
            episode_lengths[i_episode] += 1
            if env.current_player == Turns[0]:
                #move = eps_greedy(Q, str(grid), grid, epsilon=epsilon - epsilon/num_episodes*i_episode)
                
                #Remark: for question 2.1 epsilon needs to be constant and not decreasing
                move = eps_greedy(Q, str(grid), grid, epsilon = epsilon)
            else:  
                move = player.act(grid)
            
            new_grid, end, winner = env.step(move, print_grid=False)
            episode_rewards[i_episode] += env.reward(env.current_player)
#             if end: 
#                 Q[str(grid)][move[0] * 3 + move[1]] = env.reward(Turns[0])
#             else: 
#             td_target = (discount_factor * np.max(Q[str(new_grid)])) * alpha 
#             td = (1 - alpha) * Q[str(grid)][move[0] * 3 + move[1]]
#             Q[str(grid)][move[0] * 3 + move[1]] = td + td_target 
            value = Q[str(grid)][move[0] * 3 + move[1]]
            Q[str(grid)][move[0] * 3 + move[1]] += alpha * (env.reward(Turns[0]) + discount_factor * np.max(Q[str(new_grid)]) - value)
            grid = new_grid 
    
    return Q, episode_lengths, episode_rewards

Q, episode_lengths, episode_rewards = q_learning(env, 20000)

100%|████████████████████████████████████| 20000/20000 [01:12<00:00, 276.98it/s]


In [82]:
from matplotlib import pyplot as plt

def q_eval(Q, other_player_epsilon=0.5, games=20000):
    env.reset()
    total_reward = []
    Turns = np.array(['X','O'])
    count_win = 0 
    for i in tqdm(range(games)):
        grid, end, winner = env.observe()
        Turns = Turns[np.random.permutation(2)]
        player = OptimalPlayer(epsilon=other_player_epsilon, player=Turns[1])
        while not end:
            if env.current_player == Turns[0]:
                possible_moves = np.where(np.ravel(grid) == 0)[0]
                possible = [int(elem) for elem in np.argsort(Q[str(grid)]) if elem in possible_moves]
                position = possible[-1] if possible is not [] else np.random.choice(possible_moves)
                move = (int(position / 3), position % 3) 
            else:  
                move = player.act(grid)
            grid, end, winner = env.step(move, print_grid=False)
            total_reward.append(env.reward(env.current_player))
            if winner == Turns[0]: 
                count_win += 1 
            if end:
#                 print('-------------------------------------------')
#                 print('Game end, winner is player ' + str(winner))
#                 print('Q-Learning player = ' +  Turns[0])
#                 print('Optimal player = ' +  Turns[1])
#                 env.render()
                env.reset()
                break
        
    return np.mean(total_reward), count_win / games 


reward, score = q_eval(Q, other_player_epsilon = 1., games = 20000)
print(score, reward) 

100%|███████████████████████████████████| 20000/20000 [00:16<00:00, 1190.49it/s]

0.7606 -0.14894424563523415


In [ ]:
plt.plot(episode_rewards); 

## DQN 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import namedtuple
from itertools import count
import random

In [28]:
class DataSampler(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = {'state':[], 'action':[], 'next_state':[], 'reward':[]}
        self.position = 0

    def push(self, state, action, next_state, reward):
        self.memory['state'].append(state)
        self.memory['action'].append(action)
        self.memory['next_state'].append(next_state)
        self.memory['reward'].append(reward)

    def sample(self, batch_size):
        return {key: random.sample(self.memory[key], batch_size) for key in self.memory.keys()}

    def __len__(self):
        return len(self.memory)

In [29]:
class DQN(nn.Module):

    def __init__(self, states, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=1, stride=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=1, stride=1)
        self.conv3 = nn.Conv1d(32, 32, kernel_size=1, stride=1)

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
        def conv2d_size_out(size, kernel_size = 1, stride = 1):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        conv_s = conv2d_size_out(conv2d_size_out(conv2d_size_out(states)))
        linear_input_size = conv_s #* 32
        self.head = nn.Linear(32, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.reshape(-1, 1, 1)
        x = x.type(torch.float32)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return self.head(x.view(x.size(0), -1))

NameError: name 'nn' is not defined

In [30]:
BATCH_SIZE = 16
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.005
EPS_DECAY = 200
TARGET_UPDATE = 10

# Get number of actions from gym action space
n_states = 5477
n_actions = 9

policy_net = DQN(n_states, n_actions) #.to(device)
target_net = DQN(n_states, n_actions) #.to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = DataSampler(10000)

steps_done = 0

def select_action(state, grid, flag = True):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_START - EPS_START/num_episodes*(steps_done + 1)
    steps_done += 1
    if flag:
        with torch.no_grad():
            possible_moves = np.where(np.ravel(grid) == 0)[0]
            possible = [int(elem) for elem in np.argsort(np.ravel(policy_net(state).detach().numpy())) if elem in possible_moves]
            position = possible[-1] if possible is not [] else np.random.choice(possible_moves)
            return position
    # greedy 
    if sample > eps_threshold:
        with torch.no_grad():
            pos = policy_net(state).detach().numpy()
            possible_moves = np.where(np.ravel(grid) == 0)[0]
            possible = [int(elem) for elem in np.argsort(pos) if elem in possible_moves]
            position = possible[-1] if possible is not [] else np.random.choice(possible_moves) 
            return position
    else:
        return torch.tensor([[random.randrange(n_actions)]],dtype=torch.long)


episode_durations = []

NameError: name 'DQN' is not defined

In [31]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    batch = memory.sample(BATCH_SIZE)
    
    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
 
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch["next_state"])), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch["next_state"]
                                                if s is not None])
    state_batch = torch.cat(batch["state"])
    action_batch = torch.cat(batch["action"])
    reward_batch = torch.cat(batch["reward"])

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.data.clamp_(-1, 1)
    optimizer.step()

In [32]:
player = player_opt
num_episodes = 100
for i_episode in range(num_episodes):
    steps_done = i_episode
    # Initialize the environment and state
    env.reset()
    grid, end, winner = env.observe()
    for t in count():
        state = np.array([[np.ravel(grid)]])
        state = torch.from_numpy(state)
        # Resize, and add a batch dimension (BCHW)
        state = state.unsqueeze(0).type(torch.float32)
       
        # Select and perform an action
        if env.current_player == 'X':
                position = select_action(state, grid)
                move = (int(position / 3), int(position) % 3)
        else:  
                move = player.act(grid)
        
        grid, end, winner = env.step(move, print_grid=True)
        reward = torch.tensor([env.reward("X")])

        # Observe new state
        if not end:
            next_state = np.array([[np.ravel(grid)]])
            next_state = torch.from_numpy(next_state)
            next_state = next_state.unsqueeze(0).type(torch.float32)
        else:
            next_state = None
        # Store the transition in memory
        memory.push(state, [move[0] * 3 + move[1]], next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()
        if end:
            episode_durations.append(t + 1)
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

    print(f'Completed stage {i_episode+1}')

print('Complete')

NameError: name 'count' is not defined

In [33]:
def dqn_eval(Q, player=player_opt):
    total_reward = []
    env.reset()
    grid, end, winner = env.observe()
    while not end:
        if env.current_player == 'X':
            state = np.array([[grid]], dtype=np.int)
            state = torch.from_numpy(state)
            state = state.unsqueeze(0).type(torch.float32)
            position = select_action(state, grid)
            move = (int(position / 3), int(position) % 3)
        else:  
            move = player.act(grid)
        grid, end, winner = env.step(move, print_grid=False)
        total_reward.append(env.reward('X'))

        if end:
            print('-------------------------------------------')
            print('Game end, winner is player ' + str(winner))
            print('Q-Learning player = X')
            print('Another player = O')
            env.render()
            env.reset()
            break
    return np.mean(total_reward)


In [34]:
score = dqn_eval(Q)
print(score)

NameError: name 'Q' is not defined